## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [2]:
from sqlalchemy import create_engine, text, MetaData, select, func, desc
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [3]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [4]:
### Imprima os três primeiros registros da tabela tracks

session = Session(engine)

Tracks = Base.classes.tracks

query = select(Tracks).filter(Tracks.TrackId <= 3)
result = session.execute(query).scalars().all()

for track in result: #imprime cada coluna e seus valores da linha
    print({column: getattr(track, column) for column in Tracks.__table__.columns.keys()})
    
session.close()

{'TrackId': 1, 'Name': 'For Those About To Rock (We Salute You)', 'AlbumId': 1, 'MediaTypeId': 1, 'GenreId': 1, 'Composer': 'Angus Young, Malcolm Young, Brian Johnson', 'Milliseconds': 343719, 'Bytes': 11170334, 'UnitPrice': Decimal('0.99')}
{'TrackId': 2, 'Name': 'Balls to the Wall', 'AlbumId': 2, 'MediaTypeId': 2, 'GenreId': 1, 'Composer': None, 'Milliseconds': 342562, 'Bytes': 5510424, 'UnitPrice': Decimal('0.99')}
{'TrackId': 3, 'Name': 'Fast As a Shark', 'AlbumId': 3, 'MediaTypeId': 2, 'GenreId': 1, 'Composer': 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 'Milliseconds': 230619, 'Bytes': 3990994, 'UnitPrice': Decimal('0.99')}


In [5]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
session = Session(engine)

Tracks = Base.classes.tracks
Album = Base.classes.albums

# Join usando relacionamento
query = (
    select(Tracks, Album)
    .join(Album, Tracks.AlbumId == Album.AlbumId)  # Cláusula manual de join
    .order_by(Tracks.TrackId)
    .filter(Tracks.TrackId <= 20)
)
result = session.execute(query).all()

for track, album in result:
    print(f"{track.TrackId} -> Track: {track.Name}, Album: {album.Title}")


1 -> Track: For Those About To Rock (We Salute You), Album: For Those About To Rock We Salute You
2 -> Track: Balls to the Wall, Album: Balls to the Wall
3 -> Track: Fast As a Shark, Album: Restless and Wild
4 -> Track: Restless and Wild, Album: Restless and Wild
5 -> Track: Princess of the Dawn, Album: Restless and Wild
6 -> Track: Put The Finger On You, Album: For Those About To Rock We Salute You
7 -> Track: Let's Get It Up, Album: For Those About To Rock We Salute You
8 -> Track: Inject The Venom, Album: For Those About To Rock We Salute You
9 -> Track: Snowballed, Album: For Those About To Rock We Salute You
10 -> Track: Evil Walks, Album: For Those About To Rock We Salute You
11 -> Track: C.O.D., Album: For Those About To Rock We Salute You
12 -> Track: Breaking The Rules, Album: For Those About To Rock We Salute You
13 -> Track: Night Of The Long Knives, Album: For Those About To Rock We Salute You
14 -> Track: Spellbound, Album: For Those About To Rock We Salute You
15 -> Track

In [6]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
session = Session(engine)

Tracks = Base.classes.tracks
Invoice_itens = Base.classes.invoice_items

# Join usando relacionamento
query = (
    select(Tracks, Invoice_itens)
    .join(Invoice_itens, Tracks.TrackId == Invoice_itens.TrackId)  # Cláusula manual de join
    .order_by(Tracks.TrackId)
    .filter(Invoice_itens.TrackId <= 10)
)
result = session.execute(query).all()

for track, invoice in result:
    print(f"{track.TrackId} -> Track: {track.Name}, Quantidade: {invoice.Quantity}")

1 -> Track: For Those About To Rock (We Salute You), Quantidade: 1
2 -> Track: Balls to the Wall, Quantidade: 1
2 -> Track: Balls to the Wall, Quantidade: 1
3 -> Track: Fast As a Shark, Quantidade: 1
4 -> Track: Restless and Wild, Quantidade: 1
5 -> Track: Princess of the Dawn, Quantidade: 1
6 -> Track: Put The Finger On You, Quantidade: 1
8 -> Track: Inject The Venom, Quantidade: 1
8 -> Track: Inject The Venom, Quantidade: 1
9 -> Track: Snowballed, Quantidade: 1
9 -> Track: Snowballed, Quantidade: 1
10 -> Track: Evil Walks, Quantidade: 1


In [7]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
session = Session(engine)

Tracks = Base.classes.tracks
Invoice_itens = Base.classes.invoice_items

query = (
    select(Tracks.Name, Invoice_itens.TrackId, func.count(Invoice_itens.Quantity).label("total_sales"))\
    .join(Invoice_itens, Tracks.TrackId == Invoice_itens.TrackId)\
    .group_by(Invoice_itens.TrackId)\
    .order_by(desc("total_sales"))
).limit(10)
    
result = session.execute(query).all()

for invoice in result:
    print(f"Track: {invoice.Name} - Vendas: {invoice.total_sales}")

Track: Balls to the Wall - Vendas: 2
Track: Inject The Venom - Vendas: 2
Track: Snowballed - Vendas: 2
Track: Overdose - Vendas: 2
Track: Deuces Are Wild - Vendas: 2
Track: Not The Doctor - Vendas: 2
Track: Por Causa De Você - Vendas: 2
Track: Welcome Home (Sanitarium) - Vendas: 2
Track: Snowblind - Vendas: 2
Track: Cornucopia - Vendas: 2


In [34]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

Tracks = Base.classes.tracks
Invoice_items = Base.classes.invoice_items
Albums = Base.classes.albums
Artists = Base.classes.artists

session = Session(engine)

# Consulta com múltiplos JOINs
query = (
    select(Albums.Title, Artists.Name, func.count(Invoice_items.Quantity).label("total_sales"))  # Contagem de vendas
    .select_from(Tracks) #tabela base para a operação
    .join(Invoice_items, Tracks.TrackId == Invoice_items.TrackId)  # Junta cada Tracks e suas vendas
    .join(Albums, Tracks.AlbumId == Albums.AlbumId)  # Junta cada track com seu album especifico
    .join(Artists, Albums.ArtistId == Artists.ArtistId)  # Junta o album com o artista que o produziu
    .group_by(Artists.Name) #Agrupa cada artista 
    .order_by(desc("total_sales"))
).limit(10)

result = session.execute(query).fetchall()

for row in result:
    print(f"Artist: {row.Name} / Vendas: {row.total_sales}")

session.close()

Artist: Iron Maiden / Vendas: 140
Artist: U2 / Vendas: 107
Artist: Metallica / Vendas: 91
Artist: Led Zeppelin / Vendas: 87
Artist: Os Paralamas Do Sucesso / Vendas: 45
Artist: Deep Purple / Vendas: 44
Artist: Faith No More / Vendas: 42
Artist: Lost / Vendas: 41
Artist: Eric Clapton / Vendas: 40
Artist: R.E.M. / Vendas: 39
